# Analyze eBPF Programs

This notebook compiles, analyzes, and generates the report for eBPF programs. 

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import system
from utils import SOFTWARE_PATH

BCC_PATH = SOFTWARE_PATH / "bcc"
BCC_TOOLS_PATH = BCC_PATH / "libbpf-tools"
BCC_OBJ_PATH = BCC_TOOLS_PATH / ".output"

TRACEE_PATH = SOFTWARE_PATH / "tracee"
TRACEE_OBJ_FILE = TRACEE_PATH / "dist" / "tracee.bpf.o"

The following cell compiles the BCC tools. 

In [3]:
system(f"make -C {BCC_TOOLS_PATH} -j $(nproc)", linux=True)

[       system.py:15 ] INFO: Running command: "orbctl run bash -c 'make -C /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools -j $(nproc)'"


make: Entering directory '/Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools'
make: Nothing to be done for 'all'.
make: Leaving directory '/Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools'


The following cell compiles the eBPF program for Tracee.

In [4]:
system(f"make -C {TRACEE_PATH} bpf -j $(nproc)", linux=True)

[       system.py:15 ] INFO: Running command: "orbctl run bash -c 'make -C /Users/szhong/Code/DepSurf/data/software/tracee bpf -j $(nproc)'"


make: Entering directory '/Users/szhong/Code/DepSurf/data/software/tracee'
make: Nothing to be done for 'bpf'.
make: Leaving directory '/Users/szhong/Code/DepSurf/data/software/tracee'


The following cell analyzes and generates the report for eBPF programs.

In [6]:
from depsurf import BPFProgram, VersionGroup, DepReport, DepKind
from utils import OUTPUT_PATH, save_pkl

GROUPS = [VersionGroup.REGULAR, VersionGroup.ARCH]
PROG_PATHS = sorted(BCC_OBJ_PATH.glob("*.bpf.o")) + [TRACEE_OBJ_FILE]


REPORT_KINDS = [
    DepKind.FUNC,
    DepKind.TRACEPOINT,
    DepKind.LSM,
    DepKind.FIELD,
    DepKind.STRUCT,
    DepKind.SYSCALL,
]

results = {}
for path in PROG_PATHS:
    bpf = BPFProgram(path)
    result_path = OUTPUT_PATH / "programs" / bpf.name
    result_path.mkdir(parents=True, exist_ok=True)
    prog_report = {}
    for dep in bpf.deps:
        if dep.kind not in REPORT_KINDS:
            continue
        with open(result_path / f"{dep.kind} {dep.name}.md", "w") as f:
            report = DepReport.from_groups(dep, GROUPS)
            report.print(file=f)
            prog_report[dep] = report
    print(f"Report saved to {result_path}", flush=True)
    results[bpf.name] = prog_report


save_pkl(results, "bcc")

[       system.py:15 ] INFO: Running command: "orbctl run bash -c '/Users/szhong/Code/DepSurf/depsurf/btf/bpftool/src/bpftool gen min_core_btf /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools/.output/bashreadline.bpf.o /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools/.output/bashreadline.bpf.min.btf.tmp /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools/.output/bashreadline.bpf.o'"
[       system.py:15 ] INFO: Running command: "orbctl run bash -c '/Users/szhong/Code/DepSurf/depsurf/btf/bpftool/src/bpftool btf dump file /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools/.output/bashreadline.bpf.min.btf --json > /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools/.output/bashreadline.bpf.min.btf.json.tmp'"


Report saved to /Users/szhong/Code/DepSurf/data/output/programs/bashreadline


[       system.py:15 ] INFO: Running command: "orbctl run bash -c '/Users/szhong/Code/DepSurf/depsurf/btf/bpftool/src/bpftool gen min_core_btf /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools/.output/bindsnoop.bpf.o /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools/.output/bindsnoop.bpf.min.btf.tmp /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools/.output/bindsnoop.bpf.o'"
libbpf: prog '.text': relo #3: field offset ambiguity: 0 != 6338
libbpf: prog '.text': relo #4: field offset ambiguity: 0 != 6338
libbpf: prog '.text': relo #5: field offset ambiguity: 0 != 6338
libbpf: prog '.text': relo #6: field offset ambiguity: 0 != 6338
libbpf: prog '.text': relo #7: field offset ambiguity: 0 != 6338
libbpf: prog '.text': relo #8: field offset ambiguity: 0 != 6338
libbpf: prog '.text': relo #10: field offset ambiguity: 0 != 6338
libbpf: prog '.text': relo #11: field offset ambiguity: 1 != 6341
libbpf: prog '.text': relo #12: field offset ambiguity: 1 != 6341
libbpf: pro

Report saved to /Users/szhong/Code/DepSurf/data/output/programs/bindsnoop


[       system.py:15 ] INFO: Running command: "orbctl run bash -c '/Users/szhong/Code/DepSurf/depsurf/btf/bpftool/src/bpftool gen min_core_btf /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools/.output/biolatency.bpf.o /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools/.output/biolatency.bpf.min.btf.tmp /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools/.output/biolatency.bpf.o'"
libbpf: prog '.text': relo #5: field offset ambiguity: 1024 != 0
libbpf: prog '.text': relo #9: field offset ambiguity: 1024 != 0
libbpf: prog '.text': relo #21: field offset ambiguity: 1024 != 0
libbpf: prog '.text': relo #25: field offset ambiguity: 1024 != 0
libbpf: prog '.text': relo #31: field offset ambiguity: 1024 != 0
[       system.py:15 ] INFO: Running command: "orbctl run bash -c '/Users/szhong/Code/DepSurf/depsurf/btf/bpftool/src/bpftool btf dump file /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools/.output/biolatency.bpf.min.btf --json > /Users/szhong/Code/DepSurf/dat

...

[       system.py:15 ] INFO: Running command: "orbctl run bash -c '/Users/szhong/Code/DepSurf/depsurf/btf/bpftool/src/bpftool gen min_core_btf /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools/.output/wakeuptime.bpf.o /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools/.output/wakeuptime.bpf.min.btf.tmp /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools/.output/wakeuptime.bpf.o'"
[       system.py:15 ] INFO: Running command: "orbctl run bash -c '/Users/szhong/Code/DepSurf/depsurf/btf/bpftool/src/bpftool btf dump file /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools/.output/wakeuptime.bpf.min.btf --json > /Users/szhong/Code/DepSurf/data/software/bcc/libbpf-tools/.output/wakeuptime.bpf.min.btf.json.tmp'"


Report saved to /Users/szhong/Code/DepSurf/data/output/programs/wakeuptime


[       system.py:15 ] INFO: Running command: "orbctl run bash -c '/Users/szhong/Code/DepSurf/depsurf/btf/bpftool/src/bpftool gen min_core_btf /Users/szhong/Code/DepSurf/data/software/tracee/dist/tracee.bpf.o /Users/szhong/Code/DepSurf/data/software/tracee/dist/tracee.bpf.min.btf.tmp /Users/szhong/Code/DepSurf/data/software/tracee/dist/tracee.bpf.o'"
libbpf: prog 'raw_tracepoint/sys_enter_init': relo #60: field offset ambiguity: 2176 != 0
libbpf: prog 'kprobe/ptrace': relo #60: field offset ambiguity: 2176 != 0
libbpf: prog 'kprobe/process_vm_writev': relo #60: field offset ambiguity: 2176 != 0
libbpf: prog 'kprobe/arch_prctl': relo #60: field offset ambiguity: 2176 != 0
libbpf: prog 'kprobe/dup': relo #60: field offset ambiguity: 2176 != 0
libbpf: prog 'kprobe/dup2': relo #60: field offset ambiguity: 2176 != 0
libbpf: prog 'kprobe/dup3': relo #60: field offset ambiguity: 2176 != 0
libbpf: prog 'kprobe/security_file_open': relo #246: field offset ambiguity: 2176 != 0
libbpf: prog 'kpro

Report saved to /Users/szhong/Code/DepSurf/data/output/programs/tracee


[ utils_pickle.py:13 ] INFO: Saved bcc to /Users/szhong/Code/DepSurf/data/output/bcc.pkl
